In [8]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

#Memasukkan data dan menampilkan data
initial_df = pd.read_excel("Data_After_Prepro_rev.xlsx")
initial_df


,Nama_Perusahaan,Provinsi,Jenis_Produk,Sektor,Kemampuan_Produksi,Satuan_Barang,Tahun_Mulai,Pembiayaan_Bank
0,Ainul Jepara,Jawa Barat,Ukiran dan Alat Rumah Tangga,Barang Kayu dan Hasil Hutan Lainnya,20,set,1985,Ada
1,Antiq Art Shop,DKI Jakarta,Furniture,Barang Kayu dan Hasil Hutan Lainnya,150,unit,2000,Tidak Ada
2,Bayu Furniture,DKI Jakarta,"Sofa, Furniture",Barang Kayu dan Hasil Hutan Lainnya,50,set,2011,Ada
3,H. Muhidin,Jawa Barat,Sandal,Tekstil Barang Kulit dan Alas Kaki,2000,unit,2010,Tidak Ada
4,BCS,Jawa Barat,Kue,Makanan dan Minuman,30,set,2011,Tidak Ada
...,...,...,...,...,...,...,...,...
94,Ukiran Mustika Jaja,DKI Jakarta,Ukiran Batu Alam,Semen dan Barang Galian,100,unit,2013,Ada
95,Whonggada,DKI Jakarta,Art Stone,Semen dan Barang Galian,50,unit,2000,Ada
96,Uniantique Lamp,Jawa Barat,Lampu Hias,Barang Lainnya,200,unit,2007,Tidak Ada
97,Yayang Parsel,DKI Jakarta,Kerajinan dari bahan klap,Barang Lainnya,100,set,1999,Ada


In [9]:
#mengambil data yang diperlukan
new_df = initial_df[["Provinsi","Sektor","Kemampuan_Produksi","Tahun_Mulai", "Pembiayaan_Bank"]]

In [10]:
#Mengubah string menjadi angka
prov_uniq = {'Jawa Barat':1, 'DKI Jakarta':2}
sek_uniq = {'Barang Kayu dan Hasil Hutan Lainnya':1,
'Tekstil Barang Kulit dan Alas Kaki':2, 'Makanan dan Minuman':3,
'Barang Lainnya':4, 'Alat Angkutan, Mesin dan Lainnya':5,
'Semen dan Barang Galian':6, 'Pupuk, Kimia dan Barang dari Karet':7,
'Logam Dasar Besi dan Baja':8}

datmin_df = new_df.copy()
datmin_df["Provinsi"] = datmin_df["Provinsi"].replace(prov_uniq)
datmin_df["Sektor"] = datmin_df["Sektor"].replace(sek_uniq)

#Menampilkan dataframe
datmin_df


,Provinsi,Sektor,Kemampuan_Produksi,Tahun_Mulai,Pembiayaan_Bank
0,1,1,20,1985,Ada
1,2,1,150,2000,Tidak Ada
2,2,1,50,2011,Ada
3,1,2,2000,2010,Tidak Ada
4,1,3,30,2011,Tidak Ada
...,...,...,...,...,...
94,2,6,100,2013,Ada
95,2,6,50,2000,Ada
96,1,4,200,2007,Tidak Ada
97,2,4,100,1999,Ada


In [11]:
#Pelatihan data
Features = ["Provinsi","Sektor","Kemampuan_Produksi","Tahun_Mulai"]
x_train = datmin_df[Features].copy()
y_train = datmin_df['Pembiayaan_Bank']
parameter = {
    "n_neighbors": [2,3,4,5,6,7,8,9,10],
    "weights": ["uniform", "distance"],
    "p": [1, 2]
    }
# knn = KNeighborsClassifier(n_neighbors=2, weights="distance", p=1)
# knn.fit(x_train,y_train)

In [12]:
from sklearn.model_selection import GridSearchCV

parameter = {
    "n_neighbors": [2,3,4,5,6,7,8,9,10],
    "weights": ["uniform", "distance"],
    "p": [1, 2]
    }

model = GridSearchCV(KNeighborsClassifier(),parameter, cv=3, n_jobs=-1, verbose=1)
model.fit(x_train,y_train)
best_param = model.best_params_

Fitting 3 folds for each of 36 candidates, totalling 108 fits


In [13]:
#Mencoba Copy ke DF baru
knn = KNeighborsClassifier(n_neighbors=best_param["n_neighbors"], weights=best_param["weights"], p=best_param["p"])
knn.fit(x_train,y_train)
data = {"Provinsi":2,
        "Sektor":2, 
        "Kemampuan_Produksi":1000,
        "Tahun_Mulai":2013}
predict_data = pd.DataFrame(data, index=[0])
datmin_df["Prediction"] = knn.predict(datmin_df[Features])
datmin_df.to_excel("Hasil_test.xlsx")


In [14]:
#pd.DataFrame(model.cv_results_).head(5)
model.best_params_

{'n_neighbors': 6, 'p': 1, 'weights': 'distance'}